In [15]:
import pandas as pd
import numpy as np
df_train = pd.read_hdf('../data/processed/train/train_case1.hdf')
df_valid = pd.read_hdf('../data/processed/validation/valid_case1.hdf')

In [16]:
df_items = pd.read_csv('../data/raw/items.csv')
df_categories = pd.read_csv('../data/raw/item_categories.csv')
df_categories['big_category'] = df_categories['item_category_name'].str.split().apply(lambda x: x[0])
df_train = pd.merge(df_train, df_items, on='item_id')
df_train = pd.merge(df_train, df_categories, on='item_category_id')

In [74]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,weekday,...,to_feb_special_days,is_nov_special_time,nov_special_gaussian,from_nov_special_days,to_nov_special_days,is_holiday,item_name,item_category_id,item_category_name,big_category
0,2013-01-02,0,59,22154,999.0,1.0,2,1,2013,2,...,51,False,0.0,88,277,False,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино
1,2013-01-23,0,24,22154,999.0,1.0,23,1,2013,2,...,30,False,0.0,109,256,False,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино
2,2013-01-20,0,27,22154,999.0,1.0,20,1,2013,6,...,33,False,0.0,106,259,False,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино
3,2013-01-02,0,25,22154,999.0,1.0,2,1,2013,2,...,51,False,0.0,88,277,False,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино
4,2013-01-03,0,25,22154,999.0,1.0,3,1,2013,3,...,50,False,0.0,89,276,False,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино


In [93]:
from dateutil.relativedelta import relativedelta
def aggregates_dates():
    now = pd.DateOffset(days=0)
    dates = {
        '1_month': (pd.DateOffset(months=1), now),
        '3_months': (pd.DateOffset(months=3), now),
        #'6_months': (pd.DateOffset(months=6), now),
        #    'year': (relativedelta(months=12), now),
        #'1_month_ago': (pd.DateOffset(months=2), pd.DateOffset(months=1))
        #    '2_quartals_ago': (relativedelta(months=9), relativedelta(months=6)),
    }
    return dates
def get_aggregates(df, df_train):
    """
    
    
    :param: df - dataframe with historical data
    :param: df_train - dataframe with columns ['shop_id', 'item_id', 'year', 'month'], 
            aggregates will be calculated for these rows
    
    """
    keys = [
        ['shop_id'],
        ['item_id'],
        ['shop_id', 'item_id']
    ]
    dates = aggregates_dates()
    retdict = dict()
    for key in keys:
        df_key = df.groupby(key + ['year', 'month'], as_index=False)[['item_cnt_day']].sum()
        df_key['date'] = pd.to_datetime(df_key['year'].astype(str) + '-' + df_key['month'].astype(str) + '-1')
        
        # calc aggregates
        df_key['cumcount_item'] = df_key.groupby(key)['item_cnt_day'].cumsum()
        df_key['date'] += pd.DateOffset(months=1)
        for date_name, date in dates.items():
            df_key['aggr_from_date'] = df_key['date'] - date[0]
            df_key['aggr_to_date'] = df_key['date'] - date[1]
            
            from_items = key + ['aggr_from_date', 'cumcount_item']
            to_items = key + ['aggr_to_date', 'cumcount_item' ]
            df_aggr = pd.merge(df_key[from_items + ['date']], df_key[to_items], 
                     left_on=key + ['aggr_from_date'], right_on=key + ['aggr_to_date'],
                    suffixes=('_from', '_to')
                    )
            df_aggr['count_aggr'] = df_aggr['cumcount_item_to'] - df_aggr['cumcount_item_from']
            df_aggr.rename(columns={'count_aggr': 'count_aggr_' + date_name}, inplace=True)
            
            df_key = pd.merge(
                df_key, df_aggr[key + ['date', 'count_aggr_' + date_name]],
                on=key + ['date']
            )
            print(df_key.columns)
            df_key.drop(['aggr_from_date', 'aggr_to_date'], axis=1, inplace=True)
        
        print(df_key.columns)
        df_key['year'], df_key['month'] = df_key['date'].dt.year, df_key['date'].dt.month
        df_key.drop(['date', 'item_cnt_day', 'cumcount_item'], axis=1, inplace=True)
        df_key.columns = [col + '_' + '_'.join(key) if col not in key + ['year', 'month'] else col for col in df_key.columns]
        
        df_train = pd.merge(df_train, df_key, on=key+['year','month'])
        
    return df_train

get_aggregates(df_train, df_train[df_train['year']==2014][['shop_id', 'item_id', 'year', 'month']].drop_duplicates())

Index(['shop_id', 'year', 'month', 'item_cnt_day', 'date', 'cumcount_item',
       'aggr_from_date', 'aggr_to_date', 'count_aggr_3_months'],
      dtype='object')
Index(['shop_id', 'year', 'month', 'item_cnt_day', 'date', 'cumcount_item',
       'count_aggr_3_months', 'aggr_from_date', 'aggr_to_date',
       'count_aggr_1_month'],
      dtype='object')
Index(['shop_id', 'year', 'month', 'item_cnt_day', 'date', 'cumcount_item',
       'count_aggr_3_months', 'count_aggr_1_month'],
      dtype='object')
Index(['item_id', 'year', 'month', 'item_cnt_day', 'date', 'cumcount_item',
       'aggr_from_date', 'aggr_to_date', 'count_aggr_3_months'],
      dtype='object')
Index(['item_id', 'year', 'month', 'item_cnt_day', 'date', 'cumcount_item',
       'count_aggr_3_months', 'aggr_from_date', 'aggr_to_date',
       'count_aggr_1_month'],
      dtype='object')
Index(['item_id', 'year', 'month', 'item_cnt_day', 'date', 'cumcount_item',
       'count_aggr_3_months', 'count_aggr_1_month'],
      dtyp

,shop_id,item_id,year,month,count_aggr_3_months_shop_id,count_aggr_1_month_shop_id,count_aggr_3_months_item_id,count_aggr_1_month_item_id,count_aggr_3_months_shop_id_item_id,count_aggr_1_month_shop_id_item_id
0,6,944,2014,1,-13080.0,-5467.0,-86.0,-24.0,-5.0,-2.0
1,25,944,2014,1,-29090.0,-12102.0,-86.0,-24.0,-6.0,-1.0
2,42,21861,2014,1,-15973.0,-6988.0,-70.0,-28.0,-4.0,-1.0
3,25,21861,2014,1,-29090.0,-12102.0,-70.0,-28.0,-8.0,-1.0
4,57,21861,2014,1,-20411.0,-9483.0,-70.0,-28.0,-6.0,-1.0
5,54,16665,2014,1,-24709.0,-10055.0,-73.0,-35.0,-3.0,-2.0
6,6,16665,2014,1,-13080.0,-5467.0,-73.0,-35.0,-3.0,-1.0
7,28,16665,2014,1,-20562.0,-8499.0,-73.0,-35.0,-4.0,-1.0
8,58,16439,2014,1,-11568.0,-5599.0,-164.0,-116.0,-4.0,-2.0
9,42,16439,2014,1,-15973.0,-6988.0,-164.0,-116.0,-5.0,-3.0


In [26]:
df = pd.DataFrame([[1,1],[1,2],[2,3]], columns=['a', 'b'])

In [27]:
df

,a,b
0,1,1
1,1,2
2,2,3


In [29]:
df.groupby('a', as_index=False).cumsum()

,b
0,1
1,3
2,3


In [30]:
df_shops_revs = df_train.groupby(['shop_id', 'year', 'month'], as_index=False)[['item_price', 'item_cnt_day']].sum()
df_shops_revs.sort_values(by=['year', 'month']).groupby('shop_id').groupby(['year', 'month'])['item_cnt_day'].cumsum()

AttributeError: Cannot access callable attribute 'groupby' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [38]:
df_shops_revs['counts'] = df_shops_revs.sort_values(by=['year', 'month']).groupby('shop_id')['item_cnt_day'].cumsum() - df_shops_revs['item_cnt_day']

/home/piotrek/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [45]:
df_shops_revs.head()

,shop_id,year,month,item_price,item_cnt_day,counts
0,0,2013,1,2.546339e+06,5578.0,0.0
1,0,2013,2,3.007530e+06,6127.0,5578.0
2,1,2013,1,1.378736e+06,2947.0,0.0
3,1,2013,2,1.547425e+06,3364.0,2947.0
4,2,2013,1,1.028861e+06,1146.0,0.0


In [70]:
df_shops_revs['date'] = pd.to_datetime(df_shops_revs['year'].astype(str) + '-' + df_shops_revs['month'].astype(str) + '-1')
df_shops_revs['date']
aa_date = relativedelta(months=2)
df_shops_revs['date'] - pd.DateOffset(months=3)

0      2012-10-01
1      2012-11-01
2      2012-10-01
3      2012-11-01
4      2012-10-01
5      2012-11-01
6      2012-12-01
7      2013-01-01
8      2013-02-01
9      2013-03-01
10     2013-04-01
11     2013-05-01
12     2013-06-01
13     2013-07-01
14     2013-08-01
15     2013-09-01
16     2013-10-01
17     2013-11-01
18     2013-12-01
19     2014-01-01
20     2014-02-01
21     2014-03-01
22     2014-04-01
23     2014-05-01
24     2014-06-01
25     2014-07-01
26     2014-08-01
27     2014-09-01
28     2014-10-01
29     2014-11-01
          ...    
1469   2012-12-01
1470   2013-01-01
1471   2013-02-01
1472   2013-03-01
1473   2013-04-01
1474   2013-05-01
1475   2013-06-01
1476   2013-07-01
1477   2013-08-01
1478   2013-09-01
1479   2013-10-01
1480   2013-11-01
1481   2013-12-01
1482   2014-01-01
1483   2014-02-01
1484   2014-03-01
1485   2014-04-01
1486   2014-05-01
1487   2014-06-01
1488   2014-07-01
1489   2014-08-01
1490   2014-09-01
1491   2014-10-01
1492   2014-11-01
1493   201